In [1]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
#from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
from keras import metrics

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import datetime as dt
import time
import glob
from IPython import display
import tensorflow as tf
tfe = tf.contrib.eager
tf.enable_eager_execution()



Using TensorFlow backend.


In [2]:
# def plot_autoencoder_outputs(encoder, n, dims):
#     #decoded_imgs = enconder.predict(x_test)
#     x_test_encoded = vae.predict(x_test, batch_size = batch_size)
#     # number of example digits to show
#     n = 5
#     plt.figure(figsize=(20, 9))
#     for i in range(n):
#         # plot original image
#         ax = plt.subplot(2, n, i + 1)
#         plt.imshow(x_test[i].reshape(*dims))
#         #plt.gray()
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)
#         if i == n/2:
#             ax.set_title('Original Images')

#         # plot reconstruction 
#         ax = plt.subplot(2, n, i + 1 + n)
#         plt.imshow(x_test_encoded[i].reshape(*dims))
#         #plt.gray()
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)
#         if i == n/2:
#             ax.set_title('Reconstructed Images')
#     plt.show()

# def plot_loss(history):
#     historydf = pd.DataFrame(history.history, index=history.epoch)
#     plt.figure(figsize=(8, 6))
#     historydf.plot(ylim=(0, historydf.values.max()))
#     plt.title('Loss: %.3f' % history.history['loss'][-1])
    
# def plot_compare_histories(history_list, name_list, plot_accuracy=True):
#     dflist = []
#     min_epoch = len(history_list[0].epoch)
#     losses = []
#     for history in history_list:
#         h = {key: val for key, val in history.history.items() if not key.startswith('val_')}
#         dflist.append(pd.DataFrame(h, index=history.epoch))
#         min_epoch = min(min_epoch, len(history.epoch))
#         losses.append(h['loss'][-1])

#     historydf = pd.concat(dflist, axis=1)

#     metrics = dflist[0].columns
#     idx = pd.MultiIndex.from_product([name_list, metrics], names=['model', 'metric'])
#     historydf.columns = idx
    
#     plt.figure(figsize=(6, 8))

#     ax = plt.subplot(211)
#     historydf.xs('loss', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
#     plt.title("Training Loss: " + ' vs '.join([str(round(x, 3)) for x in losses]))
    
#     if plot_accuracy:
#         ax = plt.subplot(212)
#         historydf.xs('acc', axis=1, level='metric').plot(ylim=(0,1), ax=ax)
#         plt.title("Accuracy")
#         plt.xlabel("Epochs")
    
#     plt.xlim(0, min_epoch-1)
#     plt.tight_layout()

In [2]:
# Import whole N-MNIST Dataset
def load_NMNIST(path):
    """
    """
    xs_train = []
    ys_train = []
    xs_test = []
    ys_test = []

    for class_index in range(0, 10):
        for (root, dirs, dat_files) in os.walk('{0}/n_Train_3/{1}'.format(path, str(class_index))):
            for file in dat_files:
                single_X = np.fromfile('{0}/n_Train_3/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32)
                xs_train.append(single_X)
                ys_train.append(class_index)

        for (root, dirs, dat_files) in os.walk('{0}/n_Test_3/{1}'.format(path, str(class_index))):
            for file in dat_files:
                xs_test.append(np.fromfile('{0}/n_Test_3/{1}/{2}'.format(path, str(class_index), file), dtype=np.int32))
                ys_test.append(class_index)

    Xtr = np.array(xs_train)
    Xtr_reshaped = Xtr.reshape(60000, 34, 34, 2)
    Ytr = np.array(ys_train)
    Xte = np.array(xs_test)
    Xte_reshaped = Xte.reshape(10000, 34, 34, 2)
    Yte = np.array(ys_test)

    return Xtr_reshaped, Ytr, Xte_reshaped, Yte

In [3]:
dataset_class_path = '/Users/ching/Desktop/Project/autoencoder_nmnist/NMNIST/datasets'
x_train_1, y_train, x_test_1, y_test = load_NMNIST(dataset_class_path)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', x_train_1.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', x_test_1.shape)
print('Test labels shape: ', y_test.shape)

# 34 x 34 x 2

Training data shape:  (60000, 34, 34, 2)
Training labels shape:  (60000,)
Test data shape:  (10000, 34, 34, 2)
Test labels shape:  (10000,)


In [4]:
# Flatten the dataset from range -1 to 1. Since the maxima and minima lies btw 15 so we divide it by 15.
x_train = (x_train_1.astype('float32'))
x_test = (x_test_1.astype('float32'))

In [5]:
np.shape(x_test)

(10000, 34, 34, 2)

In [6]:
x_train[x_train!=0]+=30
x_test[x_test!=0]+=30
#print(x_train.shape)
#print(max(x_train[0])) #Test the maximum value after flatten

In [7]:
#x_train = x_train/45
x_train[:] = [x / 45 for x in x_train]
#x_test = x_test/45
x_test[:] = [x / 45 for x in x_test]

In [8]:
# x_train[x_train>0]=1
# x_train[x_train<0]=-1
# x_test[x_test>0]=1
# x_test[x_test<0]=-1
# #print(x_train.shape)
# #print(max(x_train[0])) #Test the maximum value after flatten

In [9]:
TRAIN_BUF = 60000
BATCH_SIZE = 100

TEST_BUF = 10000

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(TRAIN_BUF).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(x_test).shuffle(TEST_BUF).batch(BATCH_SIZE)

In [17]:
class CVAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        self.inference_net = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(34, 34, 2)),
                
                tf.keras.layers.Conv2D(
                    filters=32, kernel_size=(3, 3), strides=(2, 2), activation=tf.nn.relu),
                
                tf.keras.layers.Conv2D(
                    filters=64, kernel_size=(3, 3), strides=(2, 2), activation=tf.nn.relu),
                
                tf.keras.layers.Flatten(),
                
                # No activation
                tf.keras.layers.Dense(latent_dim + latent_dim),
            ]
        )
        
        self.generative_net = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                
                tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
                
                tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
                
                tf.keras.layers.Conv2DTranspose(
                    filters=64,
                    kernel_size=(3, 3),
                    strides=(2, 2),
                    padding="SAME",
                    activation=tf.nn.relu),
                
                tf.keras.layers.Conv2DTranspose(
                    filters=32,
                    kernel_size=(3, 3),
                    strides=(2, 2),
                    padding="SAME",
                    activation=tf.nn.relu),
                
                # No activation
                tf.keras.layers.Conv2DTranspose(
                    filters=1, kernel_size=(3, 3), strides=(2, 2), padding="SAME"),
            ]
        )
        
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random_normal(shape=(100, self.latent_dim))
            return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random_normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.generative_net(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        
        return logits

In [18]:
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.log(2. * np.pi)
    return tf.reduce_sum(
        -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
        axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)
    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

def compute_gradients(model, x):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)
    return tape.gradient(loss, model.trainable_variables), loss

optimizer = tf.train.AdamOptimizer(1e-4)
def apply_gradients(optimizer, gradients, variables, global_step=None):
    optimizer.apply_gradients(zip(gradients, variables), global_step=global_step)

In [19]:
epochs = 100
latent_dim = 50
num_examples_to_generate = 16

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.
random_vector_for_generation = tf.random_normal(
    shape=[num_examples_to_generate, latent_dim])
model = CVAE(latent_dim)

In [28]:
model.fit((train_dataset, train_dataset), epochs=epochs, batch_size=BATCH_SIZE, steps_per_epoch = 60000)


ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: x=(<BatchDataset shapes: (?, 34, 34, 2), types: tf.float32>, <BatchDataset shapes: (?, 34, 34, 2), types: tf.float32>)